<a href="https://colab.research.google.com/github/NPCA-TEAM/COVID-19/blob/main/Scripts/%205%20-%20PREDICAO_MODELOS_covid_casos_ver1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Instalação, importação de bibliotecas e inicialização

##Instalação das bibliotecas

In [ ]:
#Inicia permitindo acesso ao GDrive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install pyyaml==5.4.1

In [ ]:
!pip install darts

In [ ]:
!pip install matplotlib==3.1.3  #é necessário restart runtime 

In [ ]:
!pip install pytorch-lightning

##Importação das Bibliotecas

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import pandas as pd
import numpy as np
import glob
import os
import sys
import time
import torch
import json

import matplotlib.pyplot as plt
from matplotlib.dates import DateFormatter
import matplotlib.dates as mdates

from datetime import datetime

from darts import timeseries
from darts.dataprocessing.transformers import Scaler
from darts import concatenate

from darts.models import (    
    NBEATSModel,
    TCNModel,
    TransformerModel,
    TFTModel,
    NHiTSModel
    )

from darts.metrics import mape, rmse, r2_score
from darts.utils.timeseries_generation import datetime_attribute_timeseries
from darts.utils.missing_values import fill_missing_values
from darts.models import KalmanFilter

from darts.utils.likelihood_models import (
    GaussianLikelihood, 
    QuantileRegression 
    )

import warnings
warnings.filterwarnings('ignore')
import logging
logging.disable(logging.CRITICAL)
torch.manual_seed(1); np.random.seed(1)  # for reproducibility

#Dados e campos de entrada do framework


In [ ]:
#Campos de entrada do framework
NAME_SERIES_SYMPTOM_CASES = "Casos_DataSintoma_MM_atual_PA"   #Variavel onde será adicionada o nome para a serie a ser analisada
NAME_SERIES_PUBLICATION_CASES = "Casos_Publicacao_MM_atual_PA"   #Variavel onde será adicionada o nome para a serie a ser analisada

#Número de anos para validação/teste. Similar a quantidade de dados de teste
#Se num_days_of_slice_val = 1, indica que o ultimo ano da serie será usado para testar o modelo
#Também indica a fatia usada como validação na fase de treinamento
VALIDATION_DAYS_SLICED = 7

#Número que indica o tamanho da fatia usada como treino na fase de treinamento. 
#Logo, o processo disposto neste script é baseado no metodo de janela deslizante.
WORKOUT_DAYS_SLICED = 30

#Numero de passos da previsão a frente da serie
FORECAST_DAYS = 7

# Arquivo com a base de dados
PATH_DATASET = '/content/drive/MyDrive/NPCA - COVID/_CASOS/DataSet/' 

#Pasta para salvamento do modelo
PATH_MODELS = '/content/drive/MyDrive/NPCA - COVID/_CASOS/Models/'
PATH_FIGS = '/content/drive/MyDrive/NPCA - COVID/_CASOS/Figs/'
PATH_CSV = '/content/drive/MyDrive/NPCA - COVID/_CASOS/ForecastsOutputModel/'

#Leitura do dataset

In [ ]:
#Seleciona o arquivo da base
FILE_LIST = glob.glob(PATH_DATASET + '*')

#Seleciona o ultimo arquivo que foi criado e adiciona na pasta
FILE_PATH = max(FILE_LIST, key=os.path.getctime)

#Faz leitura do arquivo excel como datafrade
DATAFRAME = pd.read_excel(FILE_PATH, sheet_name=0)

DATAFRAME

#Seleção das variáveis

In [ ]:
#Seleção das variáveis alvo
SYMPTOM_CASES = DATAFRAME.loc[:, ['data', 'Casos_DataSintoma_MM_atual_PA']]
PUBLICATION_CASES = DATAFRAME.loc[:, ['data', 'Casos_Publicacao_MM_atual_PA']]

#Seleção das variáveis auxiliares
#SYMPTOM_7DAYS_CASES = DATAFRAME.loc[:, ['data', 'Casos_DataSintoma_MM_7dias_PA']].fillna(0) 
#PUBLICATION_7DAYS_CASES = DATAFRAME.loc[:, ['data', 'Casos_Publicacao_MM_7dias_PA']].fillna(0)
SYMPTOM_14DAYS_CASES = DATAFRAME.loc[:, ['data', 'Casos_DataSintoma_MM_14dias_PA']].fillna(0)
PUBLICATION_14DAYS_CASES = DATAFRAME.loc[:, ['data', 'Casos_Publicacao_MM_14dias_PA']].fillna(0)
#OBS: No previsão historica usaremos somente os 14 Dias. 

VACCINATION_1DOSE_CASES = DATAFRAME.loc[:, ['data', 'Vacinacao_Dose1_%decimal_PA']].fillna(0)
VACCINATION_2DOSE_CASES = DATAFRAME.loc[:, ['data', 'Vacinacao_Dose2_%decimal_PA']].fillna(0)
VACCINATION_3DOSE_CASES = DATAFRAME.loc[:, ['data', 'Vacinacao_Dose3_%decimal_PA']].fillna(0)

#Renomeia variaveis.
SYMPTOM_7PREVIOUSDAYS_CASES = SYMPTOM_14DAYS_CASES.rename(columns = {'Casos_DataSintoma_MM_14dias_PA':'Casos_DataSintoma_MM_7diasAnteriores_PA'}, inplace = False)
PUBLICATION_7PREVIOUSDAYS_CASES = PUBLICATION_14DAYS_CASES.rename(columns = {'Casos_Publicacao_MM_14dias_PA':'Casos_Publicacao_MM_7diasAnteriores_PA'}, inplace = False)

#Instanciação das séries temporais

In [ ]:
# criação das séries temporais alvos
series_symptom = timeseries.TimeSeries.from_dataframe(df=SYMPTOM_CASES, time_col='data')
series_publication = timeseries.TimeSeries.from_dataframe(df=PUBLICATION_CASES, time_col='data')

plt.figure(figsize = (14, 10))

series_symptom.plot()
series_publication.plot()

In [ ]:
# criação das séries temporais auxiliares de casos
#series_symptom_7days = timeseries.TimeSeries.from_dataframe(df=SYMPTOM_7DAYS, time_col='data')
#series_publication_7days = timeseries.TimeSeries.from_dataframe(df=PUBLICATION_7DAYS, time_col='data')
series_symptom_7previousdays = timeseries.TimeSeries.from_dataframe(df=SYMPTOM_7PREVIOUSDAYS_CASES, time_col='data')
series_publication_7previousdays = timeseries.TimeSeries.from_dataframe(df=PUBLICATION_7PREVIOUSDAYS_CASES, time_col='data')

plt.figure(figsize = (14, 10))

#series_symptom_7days.plot()
#series_publication_7days.plot()
series_symptom_7previousdays.plot()
series_publication_7previousdays.plot()

In [ ]:
# criação das séries temporais auxiliares de vacinação
series_vaccination_1Dose = timeseries.TimeSeries.from_dataframe(df=VACCINATION_1DOSE_CASES, time_col='data')
series_vaccination_2Dose = timeseries.TimeSeries.from_dataframe(df=VACCINATION_2DOSE_CASES, time_col='data')
series_vaccination_3Dose = timeseries.TimeSeries.from_dataframe(df=VACCINATION_3DOSE_CASES, time_col='data')

plt.figure(figsize = (14, 10))
  
series_vaccination_1Dose.plot()
series_vaccination_2Dose.plot()
series_vaccination_3Dose.plot()

#Filtro de suaviação das variáveis

In [ ]:
def filter_for_suavization(serie):
  filterKalman = KalmanFilter(dim_x = 1)
  filterKalman.fit(serie)
  serieTemp_filtered = filterKalman.filter(serie)

  return serieTemp_filtered

In [ ]:
#Suavização das series alvos
smoothed_symptom_series = filter_for_suavization(series_symptom)
smoothed_publication_series = filter_for_suavization(series_publication)

plt.figure(figsize = (14, 10))

series_symptom.plot(label="Sintoma")
smoothed_symptom_series.plot(label="Sintoma Suavizada")
series_publication.plot(label="Publicação")
smoothed_publication_series.plot(label="Publicação Suavizada")

#Escalar (normalização entre 0 e 1)

In [ ]:
def process_scaler(serie):
  #Pré-processamento - scalar (normalizar entre 0 e 1)
  Scaler_obj = Scaler()
  serieTemp_scaled = Scaler_obj.fit_transform(serie)

  return serieTemp_scaled, Scaler_obj
  #Retorno: Objeto Scaler da série, objeto scaler da covariável, serie temporal já normalizada

In [ ]:
#pré-processamento e normalização [0, 1]
normalized_symptom_series, Scaler_symptom = process_scaler(smoothed_symptom_series)
normalized_publication_series, Scaler_publication = process_scaler(smoothed_publication_series)

#normalized_7days_symptom_series, Scaler_symptom_7days = process_scaler(series_symptom_7days)
#normalized_7days_publication_series, Scaler_publication_7days = process_scaler(series_publication_7days)

symptom_series_7days_ago_normalized, Scaler_symptom_14days = process_scaler(series_symptom_7previousdays)
publication_series_7days_ago_normalized, Scaler_publication_14days = process_scaler(series_publication_7previousdays)

In [ ]:
plt.figure(figsize = (14, 10))

normalized_symptom_series.plot()
normalized_publication_series.plot()

#normalized_7days_symptom_series.plot()
#normalized_7days_publication_series.plot()

symptom_series_7days_ago_normalized.plot()
publication_series_7days_ago_normalized.plot()

#Definição de Covariáveis

In [ ]:
def define_covariate_dates(serie_temp_normalizada):  #modificar nomes
  #Definir covariavel ano
  scaler_CovarYear, scaler_CovarMonth = Scaler(), Scaler()
  covariate_y = datetime_attribute_timeseries(serie_temp_normalizada, attribute='year', add_length = FORECAST_DAYS)
  covariate_y = scaler_CovarYear.fit_transform(covariate_y)

  #Definir covariavel mes
  covariate_m = datetime_attribute_timeseries(serie_temp_normalizada, attribute='month', add_length = FORECAST_DAYS)
  covariate_m = scaler_CovarMonth.fit_transform(covariate_m)

  #Definir covariavel day of week
  covariate_dw = datetime_attribute_timeseries(serie_temp_normalizada, attribute='dayofweek', add_length = FORECAST_DAYS)
  covariate_dw = scaler_CovarMonth.fit_transform(covariate_dw)

  return covariate_y, covariate_m, covariate_dw

In [ ]:
#definição das covariáveis
covariate_year_symptom, covariate_month_symptom, covariate_dayweek_symptom = define_covariate_dates(normalized_symptom_series)

In [ ]:
#Visualização das series covariaveis de data
plt.figure(figsize = (14, 10))
covariate_year_symptom.plot(label='Covariates by Year')
covariate_month_symptom.plot(label='Covariates by Month')


plt.title(label='Series of Covariates by Month and Year (Cases)', fontsize=16)
plt.savefig(PATH_FIGS + 'Series of Covariates by Month and Year (Cases).png', dpi=300)

In [ ]:
plt.figure(figsize = (14, 10))
covariate_dayweek_symptom[-30:].plot(label='Day of Week' ,marker='o')


plt.title(label='Series of Covariates by Days of The Week', fontsize=16)
plt.savefig(PATH_FIGS + 'Series of Covariates by Days of The Week.png', dpi=300)

#Divisão do dataset de treino e validação

In [ ]:
#Dividir em dados de treino e teste
def split_train_val_series(serie):
  s_train, s_val = serie[:-VALIDATION_DAYS_SLICED], serie[-VALIDATION_DAYS_SLICED:]
  
  return s_train, s_val

In [ ]:
#divisão dos dados de treino e testes
serieTrain_symptom, serieVal_symptom = split_train_val_series(normalized_symptom_series)
serieTrain_publication, serieVal_publication = split_train_val_series(normalized_publication_series)

In [ ]:
#Visualização das Series Alvos
plt.figure(figsize = (14, 10))

serieTrain_symptom[-30:].plot()   # plotagem dos ultimos 30 dias para treino
serieVal_symptom.plot()
serieTrain_publication[-30:].plot()   # plotagem dos ultimos 30 dias para treino
serieVal_publication.plot()


#Predição dos modelos

##Preparação para predição

In [ ]:
# Utilizar o acelerador de GPU do colab (para treinamento)
import tensorflow as tf

device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  print('GPU device not found')
else:
  print('Found GPU at: {}'.format(device_name))
  tf.device('/device:GPU:0')
  print()

  physical_devices = tf.config.experimental.list_logical_devices('GPU')
  print("GPUs available: ", physical_devices)
  print("GPUs available: ", len(physical_devices))
  print(physical_devices[0].name)
  print()

  # Qual placa de video GPU estou usando?
  from tensorflow.python.client import device_lib
  device_lib.list_local_devices()
  print(device_lib.list_local_devices())

In [ ]:
#Junção das variaveis alvo e coovariaveis
series_Train = serieTrain_symptom.stack(serieTrain_publication)

covariatesDate = covariate_year_symptom.stack(covariate_month_symptom.stack(covariate_dayweek_symptom))
covariatesCases = symptom_series_7days_ago_normalized.stack(publication_series_7days_ago_normalized)
covariatesVacination = series_vaccination_1Dose.stack(series_vaccination_2Dose.stack(series_vaccination_3Dose))

#O [:-FORECAST_DAYS] é um corte na covariatesDate para que as covariaveis tenham o mesmo numero de dias.
covariates = covariatesDate[:-FORECAST_DAYS].stack(covariatesCases.stack(covariatesVacination))

plt.figure(figsize = (14, 10))
series_Train.plot()
covariates.plot()

##Importação dos modelos para predição

In [ ]:
#Seleciona os arquivos 
file_list_models = glob.glob(PATH_MODELS + '*.tar')
#print(file_list_models,'\n -----')

#Seleciona os arquivos adicionados mais recentementes na pasta
file_path_models = sorted(file_list_models, key=os.path.getctime, reverse=True)
#print(*file_path_models,sep='\n')

In [ ]:
#Filtrar caminhos dos modelos que tem extensão pth.tar validas
current_year = str(datetime.today().year)
file_path_models_currents = list(filter(lambda x: current_year+'.pth.tar' in x, file_path_models))
print(*file_path_models_currents,sep='\n')

In [ ]:
#Selecionar o caminho dos modelos
NBEATS_model_path = list(filter(lambda x: 'NBEATS' in x, file_path_models_currents))[0]
model_NBEATSModel = NBEATSModel.load_model(NBEATS_model_path)
print(NBEATS_model_path)

TCN_model_path = list(filter(lambda x: 'TCN' in x, file_path_models_currents))[0]
model_TCNModel = TCNModel.load_model(TCN_model_path)
print(TCN_model_path)

TFT_model_path = list(filter(lambda x: 'TFT' in x, file_path_models_currents))[0]
model_TFTModel = TFTModel.load_model(TFT_model_path)
print(TFT_model_path)

TRANSFORMER_model_path = list(filter(lambda x: 'TRANSFORMER' in x, file_path_models_currents))[0]
model_TRANSFORMERModel = TransformerModel.load_model(TRANSFORMER_model_path)
print(TRANSFORMER_model_path)

NHITS_model_path = list(filter(lambda x: 'NHITS' in x, file_path_models_currents))[0]
model_NHITSModel = NHiTSModel.load_model(NHITS_model_path)
print(NHITS_model_path)

##Predição sobre dados de treino

In [ ]:
from timeit import default_timer as timer
start = timer()

In [ ]:
def historical_forecasts(model, series_target, covariates):
  #print(series_target.components)
  #print(covariates.components)
  serie_hist_fcast = model.historical_forecasts(
      series = series_target,
      past_covariates = covariates,
      start = int(round(len(series_target)*0.05,0)), #WORKOUT_DAYS_SLICED,
      forecast_horizon = VALIDATION_DAYS_SLICED,
      stride = 1,
      num_samples = 50,  #pode ser reduzido para teste. valor padrão: 50
      retrain = False,
      verbose = False
      )  
  
  return serie_hist_fcast

In [ ]:
historical_forecasts_NBEATS = historical_forecasts(model_NBEATSModel, series_Train, covariates)

In [ ]:
historical_forecasts_TCN = historical_forecasts(model_TCNModel, series_Train, covariates)

In [ ]:
historical_forecasts_TFT = historical_forecasts(model_TFTModel, series_Train, covariates)

In [ ]:
historical_forecasts_Transformer = historical_forecasts(model_TRANSFORMERModel, series_Train, covariates)

In [ ]:
historical_forecasts_NHITS = historical_forecasts(model_NHITSModel, series_Train, covariates)

##Predição sobre dados de validação

In [ ]:
def serie_predict(model, series_target, covariates):
  serie_predicted = model.predict(
      n = FORECAST_DAYS,
      series = series_target,
      past_covariates = covariates,
      num_samples = 50,
      n_jobs = -1,
      verbose = False
      ) 
  
  return serie_predicted

In [ ]:
serie_predict_NBEATS = serie_predict(model_NBEATSModel, series_Train, covariates)

In [ ]:
serie_predict_TCN = serie_predict(model_TCNModel, series_Train, covariates)

In [ ]:
serie_predict_TFT = serie_predict(model_TFTModel, series_Train, covariates)

In [ ]:
serie_predict_Transformer = serie_predict(model_TRANSFORMERModel, series_Train, covariates)

In [ ]:
serie_predict_NHITS = serie_predict(model_NHITSModel, series_Train, covariates)

In [ ]:
end = timer()
print("Tempo decorrido de predição: {:.2f}".format(end - start))

##Visualização das predições


In [ ]:
#Visualizar series alvos e predições
plt.figure(figsize = (14, 10))
series_Train.plot()
serieVal_symptom.plot(label ='serieVal_sintoma - Real')
serieVal_publication.plot(label ='serieVal_publicacao - Real')

historical_forecasts_NBEATS.plot()
serie_predict_NBEATS.plot()

In [ ]:
#Visualizar series alvos e predições
plt.figure(figsize = (14, 10))
series_Train[-30:].plot()
serieVal_symptom.plot(label ='serieVal_sintoma - Real')
serieVal_publication.plot(label ='serieVal_publicacao - Real')

historical_forecasts_NBEATS[-30:].plot()
serie_predict_NBEATS.plot()

In [ ]:
#Visualizar series alvos e predições
plt.figure(figsize = (14, 10))
series_Train.plot()
serieVal_symptom.plot(label ='serieVal_sintoma - Real')
serieVal_publication.plot(label ='serieVal_publicacao - Real')

historical_forecasts_TCN.plot()
serie_predict_TCN.plot()

In [ ]:
#Visualizar series alvos e predições
plt.figure(figsize = (14, 10))
series_Train[-30:].plot()
serieVal_symptom.plot(label ='serieVal_sintoma - Real')
serieVal_publication.plot(label ='serieVal_publicacao - Real')

historical_forecasts_TCN[-30:].plot()
serie_predict_TCN.plot()

In [ ]:
#Visualizar series alvos e predições
plt.figure(figsize = (14, 10))
series_Train.plot()
serieVal_symptom.plot(label ='serieVal_sintoma - Real')
serieVal_publication.plot(label ='serieVal_publicacao - Real')

historical_forecasts_TFT.plot()
serie_predict_TFT.plot()

In [ ]:
#Visualizar series alvos e predições
plt.figure(figsize = (14, 10))
series_Train[-30:].plot()
serieVal_symptom.plot(label ='serieVal_sintoma - Real')
serieVal_publication.plot(label ='serieVal_publicacao - Real')

historical_forecasts_TFT[-30:].plot()
serie_predict_TFT.plot()

In [ ]:
#Visualizar series alvos e predições
plt.figure(figsize = (14, 10))
series_Train.plot()
serieVal_symptom.plot(label ='serieVal_sintoma - Real')
serieVal_publication.plot(label ='serieVal_publicacao - Real')

historical_forecasts_Transformer.plot()
serie_predict_Transformer.plot()

In [ ]:
  #Visualizar series alvos e predições
plt.figure(figsize = (14, 10))
series_Train[-30:].plot()
serieVal_symptom.plot(label ='serieVal_sintoma - Real')
serieVal_publication.plot(label ='serieVal_publicacao - Real')

historical_forecasts_Transformer[-30:].plot()
serie_predict_Transformer.plot()

#Geração e Visualização da mediana das series

In [ ]:
def series_view(historical_forecasts, serie_predict):

  median_histfcast_symptom = historical_forecasts['Casos_DataSintoma_MM_atual_PA'].median()
  median_histfcast_publication = historical_forecasts['Casos_Publicacao_MM_atual_PA'].median()
  #median_histfcast_symptom = historical_forecasts['0'].median()
  #median_histfcast_publication = historical_forecasts['1'].median()

  median_symptom_prediction = serie_predict['Casos_DataSintoma_MM_atual_PA'].median()
  median_publication_prediction = serie_predict['Casos_Publicacao_MM_atual_PA'].median()

  return median_histfcast_symptom, median_histfcast_publication, median_symptom_prediction, median_publication_prediction

In [ ]:
NBEATS_median_histfcast_symptom, NBEATS_median_histfcast_publication, NBEATS_median_symptom_prediction, NBEATS_median_publication_prediction = series_view(historical_forecasts_NBEATS, serie_predict_NBEATS)

#Aumento do tamanho da figura (x, y)
plt.figure(figsize = (14, 10))

NBEATS_median_histfcast_symptom.plot(label='NBEATS_Mediana_Previsão_Histórica Casos_DataSintoma_MM_atual_PA')
NBEATS_median_histfcast_publication.plot(label='NBEATS_Mediana_Previsão_Histórica Casos_Publicacao_MM_atual_PA')
NBEATS_median_symptom_prediction.plot()
NBEATS_median_publication_prediction.plot()

In [ ]:
TCN_median_histfcast_symptom, TCN_median_histfcast_publication, TCN_median_symptom_prediction, TCN_median_publication_prediction = series_view(historical_forecasts_TCN, serie_predict_TCN)

#Aumento do tamanho da figura (x, y)
plt.figure(figsize = (14, 10))

TCN_median_histfcast_symptom.plot(label='TCN_Mediana_Previsão_Histórica Casos_DataSintoma_MM_atual_PA')
TCN_median_histfcast_publication.plot(label='TCN_Mediana_Previsão_Histórica Casos_DataSintoma_MM_atual_PA')
TCN_median_symptom_prediction.plot()
TCN_median_publication_prediction.plot()

In [ ]:
TFT_median_histfcast_symptom, TFT_median_histfcast_publication, TFT_median_symptom_prediction, TFT_median_publication_prediction = series_view(historical_forecasts_TFT, serie_predict_TFT)

#Aumento do tamanho da figura (x, y)
plt.figure(figsize = (14, 10))

TFT_median_histfcast_symptom.plot(label='TFT_Mediana_Previsão_Histórica Casos_DataSintoma_MM_atual_PA')
TFT_median_histfcast_publication.plot(label='TFT_Mediana_Previsão_Histórica Casos_DataSintoma_MM_atual_PA')
TFT_median_symptom_prediction.plot()
TFT_median_publication_prediction.plot()

In [ ]:
Transformer_median_histfcast_symptom, Transformer_median_histfcast_publication, Transformer_median_symptom_prediction, Transformer_median_publication_prediction  = series_view(historical_forecasts_Transformer, serie_predict_Transformer)

#Aumento do tamanho da figura (x, y)
plt.figure(figsize = (14, 10))

Transformer_median_histfcast_symptom.plot(label='TRANSFORMER_Mediana_Previsão_Histórica Casos_DataSintoma_MM_atual_PA')
Transformer_median_histfcast_publication.plot(label='TRANSFORMER_Mediana_Previsão_Histórica Casos_DataSintoma_MM_atual_PA')
Transformer_median_symptom_prediction.plot()
Transformer_median_publication_prediction.plot()

In [ ]:
NHITS_median_histfcast_symptom, NHITS_median_histfcast_publication, NHITS_median_symptom_prediction, NHITS_median_publication_prediction = series_view(historical_forecasts_NHITS, serie_predict_NHITS)

#Aumento do tamanho da figura (x, y)
plt.figure(figsize = (14, 10))

NHITS_median_histfcast_symptom.plot(label='NHITS_Mediana_Previsão_Histórica Casos_DataSintoma_MM_atual_PA')
NHITS_median_histfcast_publication.plot(label='NHITS_Mediana_Previsão_Histórica Casos_DataSintoma_MM_atual_PA')
NHITS_median_symptom_prediction.plot()
NHITS_median_publication_prediction.plot()

#Re-escala dos dados gerados




In [ ]:
def process_re_scaling(Scaler_serie, serie_scaled): 
  #Volta para a escala original
  serie_back = Scaler_serie.inverse_transform(serie_scaled)  

  return serie_back

In [ ]:
# Re-escala NBEATS
NBEATS_publication_serie_histfcast_rescaled = process_re_scaling(Scaler_publication, NBEATS_median_histfcast_publication)
NBEATS_publication_serie_prediction_rescaled = process_re_scaling(Scaler_publication, NBEATS_median_publication_prediction) 
NBEATS_sympthom_serie_histfcast_rescaled = process_re_scaling(Scaler_symptom, NBEATS_median_histfcast_symptom)
NBEATS_sympthom_serie_prediction_rescaled = process_re_scaling(Scaler_symptom, NBEATS_median_symptom_prediction)

# Re-escala TCN
TCN_publication_serie_histfcast_rescaled = process_re_scaling(Scaler_publication, TCN_median_histfcast_publication)
TCN_publication_serie_prediction_rescaled = process_re_scaling(Scaler_publication, TCN_median_publication_prediction)
TCN_sympthom_serie_histfcast_rescaled = process_re_scaling(Scaler_symptom, TCN_median_histfcast_symptom)
TCN_sympthom_serie_prediction_rescaled = process_re_scaling(Scaler_symptom, TCN_median_symptom_prediction)

# Re-escala TFT
TFT_publication_serie_histfcast_rescaled = process_re_scaling(Scaler_publication, TFT_median_histfcast_publication)
TFT_publication_serie_prediction_rescaled = process_re_scaling(Scaler_publication, TFT_median_publication_prediction)
TFT_sympthom_serie_histfcast_rescaled = process_re_scaling(Scaler_symptom, TFT_median_histfcast_symptom)
TFT_sympthom_serie_prediction_rescaled = process_re_scaling(Scaler_symptom, TFT_median_symptom_prediction)

# Re-escala TRANSFORMER
Transformer_publication_serie_histfcast_rescaled = process_re_scaling(Scaler_publication, Transformer_median_histfcast_publication)
Transformer_publication_serie_prediction_rescaled = process_re_scaling(Scaler_publication, Transformer_median_publication_prediction)
Transformer_sympthom_serie_histfcast_rescaled = process_re_scaling(Scaler_symptom, Transformer_median_histfcast_symptom)
Transformer_sympthom_serie_prediction_rescaled = process_re_scaling(Scaler_symptom, Transformer_median_symptom_prediction)

# Re-escala NHITS
NHITS_publication_serie_histfcast_rescaled = process_re_scaling(Scaler_publication, NHITS_median_histfcast_publication)
NHITS_publication_serie_prediction_rescaled = process_re_scaling(Scaler_publication, NHITS_median_publication_prediction)
NHITS_sympthom_serie_histfcast_rescaled = process_re_scaling(Scaler_symptom, NHITS_median_histfcast_symptom)
NHITS_sympthom_serie_prediction_rescaled = process_re_scaling(Scaler_symptom, NHITS_median_symptom_prediction)

In [ ]:
title_7days = 'Prediction of moving average (7 days) of Cases'
title_37days = 'Prediction of moving average (7 days) of Cases (showing last 37 days)'

In [ ]:
#Visualização do alvo de publicação
plt.figure(figsize = (14, 10))

smoothed_publication_series.plot(label='Actual Publication')
NBEATS_publication_serie_histfcast_rescaled.plot(label='Publication Training Dataset Prediction') # Previsão publicação sobre Training Dataset
NBEATS_publication_serie_prediction_rescaled.plot(label='Publication Validation Dataset Prediction') # Previsão publicação sobre Validation Dataset

smoothed_symptom_series.plot(label='Actual Symptom')
NBEATS_sympthom_serie_histfcast_rescaled.plot(label='Prediction of First Symptoms in the Training Dataset') # Previsão primeiros sintomas sobre Training Dataset
NBEATS_sympthom_serie_prediction_rescaled.plot(label='Prediction of First Symptoms in the Validation Dataset')

plt.title(label= title_7days + ' - NBEATS Model', fontsize=16) # Previsão de Casos (média móvel 7 dias) - Modelo NBEATS

plt.savefig(PATH_FIGS + title_7days +' - NBEATS_Model.png', dpi = 300)

In [ ]:
#Visualiação com os ultimos 37 dias. - NBEATS
plt.figure(figsize = (14, 10))

smoothed_publication_series[-37:].plot(label='Actual Publication')
NBEATS_publication_serie_histfcast_rescaled[-30:].plot(label='Publication Training Dataset Prediction')
NBEATS_publication_serie_prediction_rescaled[-30:].plot(label='Publication Validation Dataset Prediction')

smoothed_symptom_series[-37:].plot(label='Actual Symptom')
NBEATS_sympthom_serie_histfcast_rescaled[-30:].plot(label='Prediction of First Symptoms in the Training Dataset')
NBEATS_sympthom_serie_prediction_rescaled[-30:].plot(label='Prediction of First Symptoms in the Validation Dataset')

plt.title(label= title_37days + ' - NBEATS Model', fontsize=16) 

plt.savefig(PATH_FIGS + title_37days + ' - NBEATS Model.png', dpi = 300)

In [ ]:
#Visualização do alvo de publicação
plt.figure(figsize = (14, 10))

smoothed_publication_series.plot(label='Actual Publication')
TCN_publication_serie_histfcast_rescaled.plot(label='Publication Training Dataset Prediction')
TCN_publication_serie_prediction_rescaled.plot(label='Publication Validation Dataset Prediction') 

smoothed_symptom_series.plot(label='Actual Symptom')
TCN_sympthom_serie_histfcast_rescaled.plot(label='Prediction of First Symptoms in the Training Dataset')
TCN_sympthom_serie_prediction_rescaled.plot(label='Prediction of First Symptoms in the Validation Dataset')

plt.title(label= title_7days + ' - TCN Model', fontsize=16) # Previsão de Casos (média móvel 7 dias) - Modelo NBEATS

plt.savefig(PATH_FIGS + title_7days +' - TCN_Model.png', dpi = 300)

In [ ]:
#Visualiação com com os ultimos 37 dias. - TCN
plt.figure(figsize = (14, 10))

smoothed_publication_series[-37:].plot(label='Actual Publication')
TCN_publication_serie_histfcast_rescaled[-30:].plot(label='Publication Training Dataset Prediction')
TCN_publication_serie_prediction_rescaled[-30:].plot(label='Publication Validation Dataset Prediction') 

smoothed_symptom_series[-37:].plot(label='Actual Symptom')
TCN_sympthom_serie_histfcast_rescaled[-30:].plot(label='Prediction of First Symptoms in the Training Dataset')
TCN_sympthom_serie_prediction_rescaled[-30:].plot(label='Prediction of First Symptoms in the Validation Dataset')

plt.title(label= title_37days + ' - TCN Model', fontsize=16)

plt.savefig(PATH_FIGS + title_37days +' - TCN_Model.png', dpi = 300)

In [ ]:
#Visualização do alvo de publicação
plt.figure(figsize = (14, 10))

smoothed_publication_series.plot(label='Actual Publication')
TFT_publication_serie_histfcast_rescaled.plot(label='Publication Training Dataset Prediction')
TFT_publication_serie_prediction_rescaled.plot(label='Publication Validation Dataset Prediction') 

smoothed_symptom_series.plot(label='Actual Symptom')
TFT_sympthom_serie_histfcast_rescaled.plot(label='Prediction of First Symptoms in the Training Dataset')
TFT_sympthom_serie_prediction_rescaled.plot(label='Prediction of First Symptoms in the Validation Dataset')

plt.title(label= title_7days + ' - TFT Model', fontsize=16)

plt.savefig(PATH_FIGS + title_7days +' - TFT_Model.png', dpi = 300)

In [ ]:
#Visualiação com com os ultimos 37 dias. - TFT
plt.figure(figsize = (14, 10))

smoothed_publication_series[-37:].plot(label='Actual Publication')
TFT_publication_serie_histfcast_rescaled[-30:].plot(label='Publication Training Dataset Prediction')
TFT_publication_serie_prediction_rescaled[-30:].plot(label='Publication Validation Dataset Prediction') 

smoothed_symptom_series[-37:].plot(label='Actual Symptom')
TFT_sympthom_serie_histfcast_rescaled[-30:].plot(label='Prediction of First Symptoms in the Training Dataset')
TFT_sympthom_serie_prediction_rescaled[-30:].plot(label='Prediction of First Symptoms in the Validation Dataset')

plt.title(label= title_37days + ' - TFT Model', fontsize=16)
plt.savefig(PATH_FIGS + title_37days +' - TFT_Model.png', dpi = 300)

In [ ]:
#Visualização do alvo de publicação
plt.figure(figsize = (14, 10))

smoothed_publication_series.plot(label='Actual Publication')
Transformer_publication_serie_histfcast_rescaled.plot(label='Publication Training Dataset Prediction')
Transformer_publication_serie_prediction_rescaled.plot(label='Publication Validation Dataset Prediction') 

smoothed_symptom_series.plot(label='Actual Symptom')
Transformer_sympthom_serie_histfcast_rescaled.plot(label='Prediction of First Symptoms in the Training Dataset')
Transformer_sympthom_serie_prediction_rescaled.plot(label='Prediction of First Symptoms in the Validation Dataset')

plt.title(label= title_7days + ' - TRANSFORMER Model', fontsize=16)

plt.savefig(PATH_FIGS + title_7days +' - TRANSFORMER_Model.png', dpi = 300)

In [ ]:
#Visualiação com com os ultimos 37 dias. - Transformer
plt.figure(figsize = (14, 10))

smoothed_publication_series[-37:].plot(label='Actual Publication')
Transformer_publication_serie_histfcast_rescaled[-30:].plot(label='Publication Training Dataset Prediction')
Transformer_publication_serie_prediction_rescaled[-30:].plot(label='Publication Validation Dataset Prediction') 

smoothed_symptom_series[-37:].plot(label='Actual Symptom')
Transformer_sympthom_serie_histfcast_rescaled[-30:].plot(label='Prediction of First Symptoms in the Training Dataset')
Transformer_sympthom_serie_prediction_rescaled[-30:].plot(label='Prediction of First Symptoms in the Validation Dataset')

plt.title(label= title_37days + ' - TRANSFORMER Model', fontsize=16)

plt.savefig(PATH_FIGS + title_37days +' - TRANSFORMER_Model.png', dpi = 300)

In [ ]:
#Visualização do alvo de publicação
plt.figure(figsize = (14, 10))

smoothed_publication_series.plot(label='Actual Publication')
NHITS_publication_serie_histfcast_rescaled.plot(label='Publication Training Dataset Prediction')
NHITS_publication_serie_prediction_rescaled.plot(label='Publication Validation Dataset Prediction') 

smoothed_symptom_series.plot(label='Actual Symptom')
NHITS_sympthom_serie_histfcast_rescaled.plot(label='Prediction of First Symptoms in the Training Dataset')
NHITS_sympthom_serie_prediction_rescaled.plot(label='Prediction of First Symptoms in the Validation Dataset')

plt.title(label= title_7days + ' - NHITS Model', fontsize=16)

plt.savefig(PATH_FIGS + title_7days + ' - NHITS_Model.png', dpi = 300)

In [ ]:
#Visualiação com com os ultimos 37 dias. - Transformer
plt.figure(figsize = (14, 10))

smoothed_publication_series[-37:].plot(label='Actual Publication')
NHITS_publication_serie_histfcast_rescaled[-30:].plot(label='Publication Training Dataset Prediction')
NHITS_publication_serie_prediction_rescaled[-30:].plot(label='Publication Validation Dataset Prediction') 

smoothed_symptom_series[-37:].plot(label='Actual Symptom')
NHITS_sympthom_serie_histfcast_rescaled[-30:].plot(label='Prediction of First Symptoms in the Training Dataset')
NHITS_sympthom_serie_prediction_rescaled[-30:].plot(label='Prediction of First Symptoms in the Validation Dataset')

plt.title(label= title_37days + ' - NHITS Model', fontsize=16)

plt.savefig(PATH_FIGS + title_37days + ' - NHITS_Model.png', dpi = 300)

#Exportação das predições

In [ ]:
def saving_templates(model, serie_S_HistCast_rescaled, serie_P_HistCast_rescaled, serie_S_Pred_rescaled, serie_P_Pred_rescaled):
  path = os.path.join(PATH_CSV, model.model_name)
  isExist = os.path.exists(path)

  if not isExist:
    os.mkdir(path)

  serie_histfcast = serie_S_HistCast_rescaled.stack(serie_P_HistCast_rescaled)
  df_serie_histfcast_toExport = serie_histfcast.pd_dataframe()
  df_serie_histfcast_toExport.reset_index(inplace=True)
  df_serie_histfcast_toExport.rename(columns = {'time':'data', '0':'Casos_DataSintoma_MM_atual_PA', '1':'Casos_Publicacao_MM_atual_PA'}, inplace=True)
  df_serie_histfcast_toExport.to_csv(path_or_buf = path +'/training_Forecast.csv',index=False)

  serie_pred_toExport = serie_S_Pred_rescaled.stack(serie_P_Pred_rescaled)
  serie_pred_toExport.to_csv(path_or_buf = path + '/validation_Forecast.csv')


In [ ]:
saving_templates(model_NBEATSModel,
    NBEATS_sympthom_serie_histfcast_rescaled, 
    NBEATS_publication_serie_histfcast_rescaled, 
    NBEATS_sympthom_serie_prediction_rescaled,
    NBEATS_publication_serie_prediction_rescaled)
print('NBEATS EXPORT')


saving_templates(model_TCNModel,
    TCN_sympthom_serie_histfcast_rescaled, 
    TCN_publication_serie_histfcast_rescaled, 
    TCN_sympthom_serie_prediction_rescaled,
    TCN_publication_serie_prediction_rescaled)
print('TCN EXPORT')


saving_templates(model_TFTModel,
    TFT_sympthom_serie_histfcast_rescaled, 
    TFT_publication_serie_histfcast_rescaled, 
    TFT_sympthom_serie_prediction_rescaled,
    TFT_publication_serie_prediction_rescaled)
print('TFT EXPORT')


saving_templates(model_TRANSFORMERModel,
    Transformer_sympthom_serie_histfcast_rescaled, 
    Transformer_publication_serie_histfcast_rescaled, 
    Transformer_sympthom_serie_prediction_rescaled,
    Transformer_publication_serie_prediction_rescaled)
print('TRANSFORMER EXPORT')


saving_templates(model_NHITSModel,
    NHITS_sympthom_serie_histfcast_rescaled,
    NHITS_publication_serie_histfcast_rescaled, 
    NHITS_sympthom_serie_prediction_rescaled,
    NHITS_publication_serie_prediction_rescaled)
print('NHITS EXPORT')

In [ ]:
#%run "/content/drive/My Drive/NPCA - COVID/Colab/TREINO E PREDIÇÃO/PREDICAO_MODELOS_covid_obitos_ver1.ipynb"